# Исследование надежности заемщиков

# Описание проекта


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [ ]:
data.isna().sum()

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [ ]:
data.isna().sum()

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
new_data = data.pivot_table(index='children', values='debt', aggfunc={'count', 'sum', 'mean'})
new_data['mean'] = new_data['mean'].apply('{:.2%}'.format)
new_data.columns=['Общее кол-во заемщиков', 'Процент задолженности по кредиту', 'Кол-во должников']
new_data

**Вывод:** Зависимость есть. Между количеством детей и процентом задолженности существует прямая зависимость. Чем больше детей у заемщика, тем выше шанс получить задолженность по кредиту. 

Важно обозначить, что у заемщиков, у которых нет ни одного ребенка, в среднем задолженность по кредиту составляет 7.54%. Вторая по величине категория - клиент с 1 ребенком. Процент задолженности по кредиту у клиентов этой категории в среднем составляет 9.23%. 

Однако следует отметить, что в семьях с тремя детьми вероятность быть в долгу у банка меньше, чем в семьях с двумя детьми. 

Мы можем заметить, что лучшие показатели выплаты по кредиту - у клиентов с пятью детьми, однако данная выборка не подходит для анализа из-за малого количества клиентов, подходящих под эту категорию.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_data = data.pivot_table(index='family_status', values='debt', aggfunc={'count', 'sum', 'mean'})
family_data['mean'] = family_data['mean'].apply('{:.2%}'.format)
family_data.columns=['Общее кол-во заемщиков', 'Процент задолженности по кредиту', 'Кол-во должников']
family_data

**Вывод:** Нельзя однозначно определить зависимость возврата кредита в срок от семейного положения заемщика.

Для сравнения возьмем три самые крупные категории клиентов:
    
    1. Не женат/не замужем (процент задолженности - 9.76%)
    2. Гражданский брак (процент задолженности - 9.31%)
    3. Женат/Замужем (процент задолженности - 7.56%)
    
Рассматривая данные три категории, можно сделать вывод, что лучше всех выплачивают по кредиту клиенты, которые находятся в законном браке. Проценты должников двух других категорий близки к друг другу, однако всё же в гражданском браке заемщики выплачивают кредит осознаннее. 

В двух категориях клиентов (***В разводе*** (процент задолженности - 7.06%) и ***Вдовец/вдова*** (процент задолженности - 6.62%)) определение зависимости затрудняется, так как количество таких клиентов мало относительно всей выборки. Важно учитывать, какие накопления присутствуют у клиента после расторжения брака или утраты близкого. Эти факторы могут повлиять на процент выплаты по кредиту.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
salary_data = data.pivot_table(index='total_income_category', values='debt', aggfunc={'count', 'sum', 'mean'})
salary_data['mean'] = salary_data['mean'].apply('{:.2%}'.format)
salary_data.columns=['Общее кол-во заемщиков', 'Процент задолженности по кредиту', 'Кол-во должников']
salary_data

**Вывод:** 
Для определения зависимости стоит рассмотреть две самые многочисленные категории заемщиков:
    
    1. Клиенты категории B (процент задолженности - 7.06%)
    2. Клиенты категории С (процент задолженности - 8.50%)
    
На основе этих данных можно отметить, что между уровнем дохода и возвратом кредита в срок присутствует прямая зависимость. У клиентов с меньшим доходом процент задолженности по кредиту больше, чем у заемщиков с большим уровнем дохода.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
point_data = data.pivot_table(index='purpose_category', values='debt', aggfunc={'count', 'sum', 'mean'})
point_data['mean'] = point_data['mean'].apply('{:.2%}'.format)
point_data.columns=['Общее кол-во заемщиков', 'Процент задолженности по кредиту', 'Кол-во должников']
point_data

**Вывод:** Показатель задолженности по кредиту у заемщиков, осуществляющих операции с недвижимостью, лучше, чем у всех других заемщиков, у которых присутствуют иные цели для взятия кредита. 

Самая большая вероятность задолженности по кредиту - у заемщиков, которые используют кредитные деньги для операций с автомобилем.

На основе данных можно предположить, что кредитные деньги, направленные на получение образования, дадут возможность повысить свой доход со временем, поэтому процент задолженности высокий относительно других жизненных целей.

Процент задолженности по кредиту у заемщиков, которые используют кредитные деньги на проведение свадьбы, в среднем составляет 7.91%.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:*
Могу привести несколько причин появления пропусков в исходных данных:

        1. Человеческий фактор
        2. Сбой системы
        3. Пользователь не указал данные по своему желанию (например: email, телефон)

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Если заполнить пропуски максимальными или минимальными значениями, то есть риск испортить исследование, поэтому правильнее заполнять медианным значением. На медиану не влияют случайные выбросы. При большом количестве данных медианное значение будет равняться среднему значению.

### Шаг 4: общий вывод.

**1. Зависимость между количеством детей и возвратом кредита в срок имеется:** Количество детей клиента влияет на факт погашения кредита в срок. Чем больше детей у клиента, тем выше риск оказаться в долгу перед банком.

***Статистика:***

1. Процент задолженности по кредиту у клиентов, не имеющих детей - 7.54%
2. Процент задолженности по кредиту у клиентов с одним ребенком - 9.23%
3. Процент задолженности по кредиту у клиентов с двумя детьми - 9.45%

**2. Зависимость между семейным положением и возвратом кредита в срок имеется:** Семейное положение влияет на факт погашения кредита тем, что у клиентов банка, находящиеся в законном или гражданском браке, показатели своевременной выплаты по кредиту лучше, чем у людей, которые никогда не были в браке.

***Статистика:***

1. Процент задолженности по кредиту у клиентов из категории "Не женат/не замужем" - 9.76%
2. Процент задолженности по кредиту у клиентов, которые состоят в гражданском браке - 9.31%
3. Процент задолженности по кредиту у клиентов, находящихся в законном браке - 7.56%

**3. Зависимость между уровнем дохода и возвратом кредита в срок имеется:** Клиенты, обладающие более высоким доходом, имеют процент задолженности по кредиту меньше, чем клиенты с меньшим уровнем дохода.

***Статистика***:

1. Процент задолженности по кредиту у клиента с категорией B (доход от 200001 до 1 млн) - 7.06%
2. Процент задолженности по кредиту у клиента с категорией С (доход от 50001 до 200 тысяч) - 8.50%

**4. Влияние цели кредита на его возврат в срок**: Самые лучшие показатели выплаты по кредиту - у клиентов, которые нуждаются в кредитных деньгах, чтобы потратить их на операции с недвижимостью, самые худшие показатели - клиенты с целью потратить кредитные деньги на операции с автомобилем. 

Процент возврата кредита в срок для клиентов с целью потратить кредитные деньги на образование обусловлен тем, что заемщик берет кредит для приобретения необходимых знаний и навыков, которые смогут повысить его доход через некоторое время. 

Процент должников, которые берут кредит для проведения свадьбы, небольшой, однако не самый лучший среди категорий, предложенных для исследования.

***Статистика***:

1. Процент задолженности по кредиту у клиента с целью кредита "операции с автомобилем" - 9.35%
2. Процент задолженности по кредиту у клиента с целью кредита "операции с недвижимостью" - 7.26%
3. Процент задолженности по кредиту у клиента с целью кредита "получение образования" - 9.25%
4. Процент задолженности по кредиту у клиента с целью кредита "проведение свадьбы" - 7.91%

**5. Рекомендации по улучшению сбора данных и скоринга**: Для более точного анализа зависимости между количеством детей и возвратом кредита в срок было бы полезным знать, сколько человек в семье имеют доход, так как один работающий человек с детьми может иметь трудности для своевременной выплаты по кредиту, нежели двое работающих людей в этой же семье.

Для более эффективного анализа зависимости между семейным положением и возвратом кредита в срок нужно уточнять у клиента, какую долю денежных средств для выплаты кредита составляют накопления у клиентов категорий "В разводе" и "Вдовец/вдова".